In [ ]:
import pandas as pd
import ast  # For safely evaluating string-formatted lists
import plotly.graph_objects as go

arenas_df = pd.read_csv("../#2 Data Storage/Utils/arenas.csv")
arenas_dict = {}

for row in arenas_df.itertuples():
    arenas_dict[row.Arena_ID] = row.Arena_Name

def check_for_card(card_list_str, card_name):
    """
    Safely parses a string representation of a card list
    and checks if a specific card is present.
    """
    try:
        card_list = ast.literal_eval(card_list_str)
        if not isinstance(card_list, list):
            return False
        for card_tuple in card_list:
            if isinstance(card_tuple, tuple) and len(card_tuple) > 0 and card_tuple[0] == card_name:
                return True
    except (ValueError, SyntaxError, TypeError):
        return False
    return False

def plot_interactive_arena_chart():
    try:
        # --- Load Troop Names ---
        troop_file_name = '../#2 Data Storage/Utils/troop_name.csv' 
        troop_df = pd.read_csv(troop_file_name)
        troop_list = troop_df['Troop_name'].unique()
        print(f"Loaded {len(troop_list)} unique troop names.")
        # Create a set for fast lookup
        troop_set = set(troop_list) 
        
        # --- Load Battle Log Data ---
        file_name = '../#2 Data Storage/Processed Data/preprocessed_battle_log.csv' 
        df = pd.read_csv(file_name)
        print("Data loaded. Processing for all troops...")

        # --- Process data for ALL troops ---
        card_data = [] # Renamed from all_card_data to match original

        for row in df.itertuples(index=False):
            # Check player 0's cards
            try:
                player_0_cards = {card[0] for card in ast.literal_eval(row.players_0_spells) if isinstance(card, tuple) and len(card) > 0}
            except (ValueError, SyntaxError, TypeError):
                player_0_cards = set()
                
            # Check player 1's cards
            try:
                player_1_cards = {card[0] for card in ast.literal_eval(row.players_1_spells) if isinstance(card, tuple) and len(card) > 0}
            except (ValueError, SyntaxError, TypeError):
                player_1_cards = set()

            # Find which troops from our list were used
            player_0_troops_used = player_0_cards.intersection(troop_set)
            player_1_troops_used = player_1_cards.intersection(troop_set)

            # Log data for player 0
            for troop_name in player_0_troops_used:
                card_data.append({
                    'arena': row.arena,
                    'outcome': 'Won' if row.players_0_winner == 1 else 'Lost',
                    'card_name': troop_name # Add the card_name column
                })
                
            # Log data for player 1
            for troop_name in player_1_troops_used:
                card_data.append({
                    'arena': row.arena,
                    'outcome': 'Won' if row.players_1_winner == 1 else 'Lost',
                    'card_name': troop_name # Add the card_name column
                })

        # --- Create and Group DataFrame ---
        if not card_data:
            print(f"No usage data found for any troops.")
        else:
            card_df = pd.DataFrame(card_data)
            
            # Group data by arena, card_name, and outcome to get counts
            # This is the key change to support the dropdown
            grouped_df = card_df.groupby(['arena', 'card_name', 'outcome']).size().reset_index(name='count')
            grouped_df['arena'] = grouped_df['arena'].map(arenas_dict)  # Map arena IDs to names
            grouped_df = grouped_df.dropna(subset=['arena'])

            # --- Sort Arenas ---
            arena_order = sorted(list(set(grouped_df['arena'])), key=lambda x: int(x.split(' ')[1]))
            grouped_df['arena'] = pd.Categorical(grouped_df['arena'], categories=arena_order, ordered=True)
            grouped_df = grouped_df.sort_values(by='arena')

            print("Data processing complete. Building plot...")

            # --- Build Plot with Dropdown ---
            # This logic was correct and remains the same
            fig = go.Figure()
            
            buttons = []
            # Get list of troops that actually have data
            troops_with_data = sorted(grouped_df['card_name'].unique())
            total_traces = len(troops_with_data) * 2 # Two traces (Won/Lost) per troop
            
            for i, troop_name in enumerate(troops_with_data):
                df_troop = grouped_df[grouped_df['card_name'] == troop_name]
                
                # Filter for Won and Lost
                df_won = df_troop[df_troop['outcome'] == 'Won']
                df_lost = df_troop[df_troop['outcome'] == 'Lost']
                
                # Set visibility: only the first troop (i=0) is visible
                visible = (i == 0)
                
                # Add 'Won' trace for this troop
                fig.add_trace(go.Bar(
                    x=df_won['arena'], y=df_won['count'], name='Won',
                    marker_color='blue', visible=visible,
                    hovertemplate=f"Card: {troop_name}<br>Arena: %{{x}}<br>Outcome: Won<br>Count: %{{y}}<extra></extra>",
                    opacity=0.5

                ))
                # Add 'Lost' trace for this troop
                fig.add_trace(go.Bar(
                    x=df_lost['arena'], y=df_lost['count'], name='Lost',
                    marker_color='red', visible=visible,
                    hovertemplate=f"Card: {troop_name}<br>Arena: %{{x}}<br>Outcome: Lost<br>Count: %{{y}}<extra></extra>",
                    opacity=0.5
                ))
                
                # Create the visibility list for the dropdown button
                visibility_list = [False] * total_traces
                visibility_list[i * 2] = True     # Set this troop's 'Won' trace to visible
                visibility_list[i * 2 + 1] = True # Set this troop's 'Lost' trace to visible
                
                # Create and add the button
                buttons.append(dict(
                    label=troop_name,
                    method="update",
                    # args[0] sets trace visibility, args[1] sets the title
                    args=[{"visible": visibility_list},
                          {"title": f"{troop_name} Usage Count per Arena by Win/Loss"}]
                ))
    
            # --- Add Layout and Dropdown ---
            fig.update_layout(
                updatemenus=[dict(
                    active=0,
                    buttons=buttons,
                    direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=0.1,
                    xanchor="left",
                    y=1.15,
                    yanchor="top"
                )],
                title_text=f"{troops_with_data[0]} Usage Count per Arena by Win/Loss", # Initial title
                xaxis_title="Arena",
                yaxis_title="Usage Count",
                barmode='overlay',
            )
            
            # Apply sorting fix
            fig.update_xaxes(categoryorder='array', categoryarray=arena_order)
            fig.show()
            
    except FileNotFoundError as e:
        print(f"Error: A required file was not found. {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# --- Run the function ---
plot_interactive_arena_chart()

Loaded 121 unique troop names.
Data loaded. Processing for all troops...
Data processing complete. Building plot...



Successfully created interactive plot with dropdown!
File saved as: 'troop_usage_per_arena_dropdown_modified.html'
